In [1]:
!pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 18.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 29.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 65.9 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.9.41
    Uninstalling nvidia-nvjitlink-cu12-12.9.41:
      Successfully uninstalled nvidia-nvjitlink-cu12-12.9.41
  Attempting uninstall: nvidia-curand-cu12
    Found existing installation: nvidia-curand-cu12 10.3.10.19
    Uninstalling nvidia-curand-cu12-10.3

In [2]:
import os

# === CẤU HÌNH ===
base_dir = "//kaggle/input/fixed-original-segment-dataset/dataset"  # hoặc thư mục chứa 'images/' và 'labels/'

def count_files(folder):
    return len([f for f in os.listdir(folder) if os.path.isfile(os.path.join(folder, f))])

def report_counts(split):
    img_dir = os.path.join(base_dir, "images", split)
    lbl_dir = os.path.join(base_dir, "labels", split)

    img_count = count_files(img_dir) if os.path.exists(img_dir) else 0
    lbl_count = count_files(lbl_dir) if os.path.exists(lbl_dir) else 0

    print(f"📂 {split.capitalize()}:")
    print(f"    Số ảnh   : {img_count}")
    print(f"    Số label : {lbl_count}")
    print(f"    Mismatch : {abs(img_count - lbl_count)}\n")

# === GỌI BÁO CÁO ===
print(" Kiểm tra số lượng ảnh và label:")
for split in ["train", "val"]:
    report_counts(split)


 Kiểm tra số lượng ảnh và label:
📂 Train:
    Số ảnh   : 1618
    Số label : 1618
    Mismatch : 0

📂 Val:
    Số ảnh   : 411
    Số label : 411
    Mismatch : 0



In [3]:
# Kiểm tra GPU
import torch
print(torch.cuda.is_available())
print(torch.cuda.get_device_name(0) if torch.cuda.is_available() else "CPU")


True
Tesla T4


In [4]:
data_yaml = """
path: /kaggle/input/fixed-original-segment-dataset/dataset
train: images/train
val: images/val
names:
  0: unbet
  1: betrivers
  2: fanduel
  3: betway
  4: caesars
  5: bally
  6: draftkings
  7: pointsbet
  8: bet365
  9: fanatics
  10: betparx
  11: betmgm
  12: gilariver
  13: casino
"""

with open("data.yaml", "w") as f:
    f.write(data_yaml)


In [5]:
from ultralytics import YOLO

# Tải model yolov8l-seg.pt
model = YOLO("Model_yolo_base/yolov8m-seg.pt")

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


100%|██████████| 52.4M/52.4M [00:00<00:00, 244MB/s]


In [6]:

model.train(
    data="data.yaml",           # sử dụng file vừa tạo
    epochs=150,
    imgsz=640,
    optimizer="Adam",
    batch=8,
    device=0,
    name="yolov8m-seg-kaggle",
)

# Nén toàn bộ folder runs vào file zip
!zip -r /kaggle/working/runs_yolo.zip /kaggle/working/runs


Ultralytics 8.3.158 🚀 Python-3.11.11 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=8, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=data.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=150, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=Model_yolo_base/yolov8m-seg.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=yolov8m-seg-kaggle, nbs=64, nms=False, opset=None, optimize=False, optimizer=Adam, overlap_mask=True, patience=100, perspective=0.0, plots=True, pose=12.0,

100%|██████████| 755k/755k [00:00<00:00, 17.0MB/s]


Overriding model.yaml nc=80 with nc=14

                   from  n    params  module                                       arguments                     
  0                  -1  1      1392  ultralytics.nn.modules.conv.Conv             [3, 48, 3, 2]                 
  1                  -1  1     41664  ultralytics.nn.modules.conv.Conv             [48, 96, 3, 2]                
  2                  -1  2    111360  ultralytics.nn.modules.block.C2f             [96, 96, 2, True]             
  3                  -1  1    166272  ultralytics.nn.modules.conv.Conv             [96, 192, 3, 2]               
  4                  -1  4    813312  ultralytics.nn.modules.block.C2f             [192, 192, 4, True]           
  5                  -1  1    664320  ultralytics.nn.modules.conv.Conv             [192, 384, 3, 2]              
  6                  -1  4   3248640  ultralytics.nn.modules.block.C2f             [384, 384, 4, True]           
  7                  -1  1   1991808  ultralytic

100%|██████████| 5.35M/5.35M [00:00<00:00, 75.7MB/s]


AMP: checks passed ✅
train: Fast image access ✅ (ping: 0.3±0.4 ms, read: 37.5±7.3 MB/s, size: 289.9 KB)


...

WARNING ⚠️ Limiting validation plots to first 50 items per image for speed...
WARNING ⚠️ Limiting validation plots to first 50 items per image for speed...
WARNING ⚠️ Limiting validation plots to first 50 items per image for speed...
WARNING ⚠️ Limiting validation plots to first 50 items per image for speed...
WARNING ⚠️ Limiting validation plots to first 50 items per image for speed...
WARNING ⚠️ Limiting validation plots to first 50 items per image for speed...
WARNING ⚠️ Limiting validation plots to first 50 items per image for speed...
WARNING ⚠️ Limiting validation plots to first 50 items per image for speed...
WARNING ⚠️ Limiting validation plots to first 50 items per image for speed...


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP50  mAP50-95):   8%|▊         | 2/26 [00:00<00:11,  2.00it/s]

WARNING ⚠️ Limiting validation plots to first 50 items per image for speed...
WARNING ⚠️ Limiting validation plots to first 50 items per image for speed...
WARNING ⚠️ Limiting validation plots to first 50 items per image for speed...
WARNING ⚠️ Limiting validation plots to first 50 items per image for speed...
WARNING ⚠️ Limiting validation plots to first 50 items per image for speed...
WARNING ⚠️ Limiting validation plots to first 50 items per image for speed...
WARNING ⚠️ Limiting validation plots to first 50 items per image for speed...


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP50  mAP50-95): 100%|██████████| 26/26 [00:07<00:00,  3.61it/s]
/usr/local/lib/python3.11/dist-packages/matplotlib/colors.py:721: RuntimeWarning: invalid value encountered in less
  xa[xa < 0] = -1
/usr/local/lib/python3.11/dist-packages/matplotlib/colors.py:721: RuntimeWarning: invalid value encountered in less
  xa[xa < 0] = -1


                   all        411       3957      0.894      0.764      0.832      0.697      0.872      0.744      0.795      0.522
                 unbet        388       3340      0.938      0.787       0.91      0.712      0.891      0.748      0.843      0.456
             betrivers         13         17      0.884      0.765      0.853       0.74      0.884      0.765      0.853      0.473
               fanduel        161        277      0.988      0.868      0.959      0.788      0.953      0.838      0.914      0.487
                betway         60         91      0.961      0.846        0.9      0.759       0.95      0.837      0.889      0.591
               caesars          4         10      0.823        0.3      0.468      0.389      0.823        0.3      0.468      0.254
                 bally         52         56      0.944      0.904      0.923      0.686      0.757      0.725      0.596      0.261
            draftkings         49         59      0.972      0.831   